<a href="https://colab.research.google.com/github/YeonJaeHong/ESAA_homeworks/blob/main/12_25_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이썬 머신러닝 7-06. 군집화 실습- 고객 세그먼테이션


고객 세그먼테이션은 CRM 이나 메케팅의 중요 기반 요소로서 다양한 기준으로 고객 분류 기법

*   고객 세그먼테이션의 주요 목표-> 타깃 마케팅 (고객을 여러 특성에 맞게 세분화해 그 유형에 따라 맞춤형 마케팅이나 서비스 제공) -고객 상품 구매 이력에서 출발

*   고객 군집화 중요: RFM 기법 이용
RFM 기법: Recency + Frequency + Monetary value


데이터 세트 로딩과 데이터 클렌징

제품 주문 데이터 세트 로딩
InvoiceNo: 주문번호, ‘C’로 시작하는 것은 취소 주문
StockCode: 제품 코드(Item Code)
Description: 제품 설명
Quantity : 주문 제품 건수
InvoiceDate: 주문 일자
UnitPrice: 제품 단가
CustomerID: 고객 번호
Country: 국가명(주문 고객의 국적)

In [14]:
import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

retail_df =pd.read_excel(io="/content/Online Retail.xlsx")

ValueError: ignored

In [5]:
retail_df.info()

NameError: ignored

null값이 많아서 사전 정제 작업이 필요하다

In [15]:
#불린 인덱싱을 적용하여 quantity > 0, unitprice > 0, customerID가  not null인 값만 필터링
retail_df = retail_df[retail_df['Quantity'] > 0]
retail_df = retail_df[retail_df['UnitPrice'] > 0]
retail_df = retail_df[retail_df['CustomerID'].notnull()]
print(retail_df.shape)
retail_df.isnull().sum()


NameError: ignored

In [ ]:
retail_df['Country'].value_counts()[:5]

In [ ]:
# 영국이 대다수를 차지하므로 다른 국가의 데이터는 모두 제외

retail_df = retail_df[retail_df['Country'] == 'United Kingdom']
print(retail_df.shape)

최종 데이터가 354321건으로 준 것을 확인



RFM 기반 데이터 가공
> unitprice * quantity = 주문 금액 데이터
> customerNo도 더 편리한 식별성을 위해 float -> int

In [16]:
retail_df['sale_amount'] = retail_df['Quantity'] * retail_df['UnitPrice']
retail_df['CustomerID'] = retail_df['CustomerID'].astype(int)

#top5 주문건수와 주문금액 추출
print(retail_df['CustomerID'].value_counts().head(5))
print(retail_df.groupby('CustomerID')['sale_amount'].sum().sort_values(ascending=False)[:5])

NameError: ignored

In [ ]:
#주문번호InvoiceNo + 상품코드StockCode 레벨의 식별자로 group by 수행시
retail_df.groupby(['InvoiceNo', 'StockCode'])['InvoiceNo'].count().mean()
#식별자 레벨이 1에 가깝게 됨

RFM 기반의 고객 세그먼테이션은 고객 레벨로 주문 기간, 주문 횟수, 주문 금액 데이터를 기반으로 해 세그먼테이션을 수행하는 것이므로 주문번호+상품코드 기준의 데이터를 고객 기준의 Recency, Frequency, Monetary value 데이터로 변경해야한다. 이를 위해서는 주문 번호 기준의 데이터를 개별 고객 기준의 데이터로 groupby 해야한다.

In [ ]:
# df의 groupby()의 multiple연산을 위해 agg() 이용
# Recency는 InvoiceDate 칼럼의 max()에서 데이터 가공
# Frequency는 InvoiceNo 칼럼의 count(), Monetary value는 sale_amount 칼럼의 sum()
aggregations = {
    'InvoiceDate':'max',
    'InvoiceNo': 'count',
    'sale_amount':'sum'
}
cust_df = retail_df.groupby('CustomerID').agg(aggregations)
# groupby된 결과 칼럼 값을 Recency, Frequency, Monetary로 변경
cust_df = cust_df.rename(columns = {'InvoiceDate':'Recency',
                                    'InvoiceNo':'Frequency',
                                    'sale_amount':'Monetary'})
cust_df = cust_df.reset_index()
cust_df.head(3)

In [ ]:
import datetime as dt

cust_df['Recency'] = dt.datetime(2011, 12, 10) - cust_df['Recency']
cust_df['Recency'] = cust_df['Recency'].apply(lambda x: x.days+1)
print('cust_df 로우와 칼럼 건수는 ', cust_df.shape)
cust_df.head(3)


RFM 기반 고객 세그먼테이션

> 온라인 판매 데이터 set는 소매업체의 대규모 주문을 포함하고있기 때문에 이로 인해 왜곡된 데이터 분포도를 가지게 되어 군집화가 한쪽 군집에만 집중되는 현상이 발생

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), nrows=1, ncols=3)
ax1.set_title('Recency Histogram')
ax1.hist(cust_df['Recency'])

ax2.set_title('Frequncy Histogram')
ax2.hist(cust_df['Frequency'])

ax3.set_title('Monetary Histogram')
ax3.hist(cust_df['Monetary'])

# 모두 왜곡된 데이터값 분포를 가지고 있으면서 특히 frequency, monetary의 경우 특정 범위에 몰려있어서 왜곡이 심하다

In [ ]:
cust_df[['Recency', 'Frequency', 'Monetary']].describe()

왜곡 정도가 매우 높은 데이터 세트에 k-평균 군집을 적용하면 중심의 개수를 증가시키더라도 변별력이 떨어지는 군집화가 수행된다, 먼저 데이터 세트를 standardscaler로 평균과 표준편차를 재조정한뒤에 k-평균을 수행하겠다

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

X_features = cust_df[['Recency', 'Frequency', 'Monetary']].values
X_features_scaled = StandardScaler().fit_transform(X_features)

kmeans = KMeans(n_clusters=3, random_state=0)
labels = kmeans.fit_predict(X_features_scaled)
cust_df['cluster_label'] = labels

print('실루엣 스코어는 : {0:.3f}'.format(silhouette_score(X_features_scaled, labels)))

In [ ]:
#개별 군집의 실루엣 계수값과 데이터 구성알아보기

def visualize_silhouette(cluster_lists, X_features):

    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math

    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)

    # plt.subplots()으로 리스트에 기재된 클러스터링 만큼의 sub figures를 가지는 axs 생성
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산.
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현.
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")



In [ ]:
#군집별로 알아본 결과
visualize_silhouette([2,3,4,5], X_features_scaled)
visualize_kmeans_plot_multi([2,3,4,5], X_features_scaled)

데이터 세트의 왜곡 정도를 낮추기 위해 로그 변환 시행

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

# RFM 칼럼에 np.log1p()로 log transformation
cust_df['Recency_log'] = np.log1p(cust_df['Recency'])
cust_df['Frequency_log'] = np.log1p(cust_df['Frequency'])
cust_df['Monetary_log'] = np.log1p(cust_df['Monetary'])

# Log Transformation 데이터에 standard scaler 적용
X_features = cust_df[['Recency_log', 'Frequency_log', 'Monetary_log']].values
X_features_scaled = StandardScaler().fit_transform(X_features)

kmeans = KMeans(n_clusters=3, random_state=0)
labels = kmeans.fit_predict(X_features_scaled)

cust_df['cluster_label'] = labels
print('실루엣 스코어는 : {0:.3f}'.format(silhouette_score(X_features_scaled, labels)))


실루엣 스코어는 로그 변환 전보다 떨어지지만, 절대치가 중요한 것이 아니다를 앞에서 확인할 수 있었음. 어떻게 개별 군집이 어떻게 더 균일하게 나뉠수 있는지가 더 중요하다.

08. 텍스트 분석

>NLP이냐 텍스트 분석이냐?

NLP는 머신이 인간의 언어를 이해하고 해석하는데 더 중점을 두고 발전했으며, 텍스트 마이닝이라고도 불리는 텍스트 분석은 비정형 텍스트에서 의미 있는 정보를 추출하는것에 더 중점을 두고 발전해 왔다.
NLP는 언어를 해석하기 위한 기계 번역, 자동으로 질문을 해석하고 답 해주는 질의응답 시스템 등의 영역에서 텍스트 분석과 차별점 있고, 텍스트 분석은 머신러닝, 언어 이해, 통계 활용해 모델 수립후 정보 추출해 bi 나 예측 분석 등 분석 작업 주로 수행한다.